In [31]:
import nltk

TRAIN_DATA = "DATA/train.csv"
TEST_DATA = "DATA/test.csv"
SUBMISSON_DATA = "DATA/submission.csv"

In [32]:
import pandas as pd
import statsmodels.api as sm

In [33]:
df = pd.read_csv(TEST_DATA)
df

,id,Содержание,Сервис,Приоритет,Статус,Функциональная группа,Крайний срок,Дата обращения,Тип обращения на момент подачи,Тип обращения итоговый,Решение,Тип переклассификации,Дата восстановления,Дата закрытия обращения,Критичность,Влияние,Система,Место
0,14803,Заявка на предоставление и отзыв прав доступа ...,Сервис41,3-Низкий,Закрыт,ФГ13,2018-02-22 07:51,2018-02-16 13:11,Запрос,NaN,Работы по обращению выполнены,NaN,2018-02-18 04:32,2018-02-25 07:56,4-Нет,4-Нет влияния,Система43,Место38
1,14804,Транспортный шлюз,Сервис61,2-Средний,Закрыт,ФГ1,2018-02-21 14:36,2018-02-16 13:13,Запрос,NaN,проинф у.о.,NaN,2018-02-16 17:55,2018-02-24 07:54,1-Особая,3-Малое,Система61,К
2,14805,Заявка на предоставление и отзыв прав доступа ...,Сервис22,3-Низкий,Закрыт,ФГ13,2018-03-31 06:36,2018-02-16 13:13,Запрос,NaN,Работы по обращению выполнены,NaN,2018-03-15 16:09,2018-03-23 07:56,2-Повышенная,4-Нет влияния,Система13,Место25
3,14806,Заявка на предоставление и отзыв прав доступа ...,Сервис91,3-Низкий,Отменен,ФГ13,2018-02-21 14:29,2018-02-16 13:14,Запрос,NaN,Запрос отменён пользователем из портала самооб...,NaN,2018-02-16 13:15,NaN,2-Повышенная,4-Нет влияния,Система363,Место1
4,14807,Заявка на предоставление и отзыв прав доступа ...,Сервис22,3-Низкий,Закрыт,ФГ13,2018-03-30 09:36,2018-02-16 13:14,Запрос,NaN,Работы по обращению выполнены,NaN,2018-03-15 16:09,2018-03-23 07:56,2-Повышенная,4-Нет влияния,Система13,Место25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6339,21142,<Бот-наблюдения> Система53. Ухудшение статуса ...,Сервис53,3-Низкий,Закрыт,ФГ4,2018-03-07 14:36,2018-02-25 20:56,Инцидент,NaN,самовосстановление,NaN,2018-02-26 05:57,2018-03-14 07:53,2-Повышенная,3-Малое,Система53,Место1
6340,21143,<Бот-наблюдения> Система52. Ухудшение статуса ...,Сервис52,2-Средний,Закрыт,ФГ4,2018-03-07 14:06,2018-02-25 20:58,Инцидент,NaN,самовосстановление,NaN,2018-02-26 06:00,2018-03-14 07:53,1-Особая,3-Малое,Система52,Место1
6341,21144,<Бот-наблюдения> Система11. Ухудшение статуса ...,Сервис11,3-Низкий,Закрыт,ФГ4,2018-03-09 12:36,2018-02-25 21:05,Инцидент,NaN,самовосстановление,NaN,2018-02-26 06:03,2018-03-14 07:53,4-Нет,3-Малое,Система11,Место1
6342,21145,<Бот-наблюдения>Система364. Ухудшение статуса ...,Сервис4,3-Низкий,Закрыт,ФГ5,2018-03-09 12:36,2018-02-25 21:16,Инцидент,NaN,Самовосстановление,NaN,2018-03-07 09:28,2018-03-14 10:43,2-Повышенная,3-Малое,Система4,Место1


In [34]:
df.rename({df.columns[9]: "Решение"}, inplace=True)
df.columns

Index(['id', 'Содержание', 'Сервис', 'Приоритет', 'Статус',
       'Функциональная группа', 'Крайний срок', 'Дата обращения',
       'Тип обращения на момент подачи', 'Тип обращения итоговый',
       'Решение             ', 'Тип переклассификации', 'Дата восстановления',
       'Дата закрытия обращения', 'Критичность', 'Влияние', 'Система',
       'Место'],
      dtype='object')

In [35]:
tdf = df[df['Статус'] == 'Отменен']
tdf

,id,Содержание,Сервис,Приоритет,Статус,Функциональная группа,Крайний срок,Дата обращения,Тип обращения на момент подачи,Тип обращения итоговый,Решение,Тип переклассификации,Дата восстановления,Дата закрытия обращения,Критичность,Влияние,Система,Место
3,14806,Заявка на предоставление и отзыв прав доступа ...,Сервис91,3-Низкий,Отменен,ФГ13,2018-02-21 14:29,2018-02-16 13:14,Запрос,NaN,Запрос отменён пользователем из портала самооб...,NaN,2018-02-16 13:15,NaN,2-Повышенная,4-Нет влияния,Система363,Место1
124,14927,Консультация,Сервис7,2-Средний,Отменен,ФГ18,2018-02-22 07:06,2018-02-16 14:51,Запрос,NaN,Запрос отменён пользователем из портала самооб...,NaN,2018-02-16 14:53,NaN,1-Особая,4-Нет влияния,Система7,Место73
164,14967,Замена/добавление Сертификат на сервере Система13,Сервис13,2-Средний,Отменен,ФГ13,2018-02-22 07:26,2018-02-16 15:11,Запрос,NaN,Запрос отменён пользователем из портала самооб...,NaN,2018-02-16 15:12,NaN,1-Особая,4-Нет влияния,Система13,Место178
234,15037,Замена/добавление Сертификат на сервере Система13,Сервис13,2-Средний,Отменен,ФГ13,2018-02-22 07:51,2018-02-17 00:34,Запрос,NaN,Запрос отменён пользователем из портала самооб...,NaN,2018-02-17 05:54,NaN,1-Особая,4-Нет влияния,Система13,Место191
278,15081,Изменение атрибутов учётной записи,Сервис7,2-Средний,Отменен,ФГ13,2018-02-22 07:51,2018-02-17 04:18,Запрос,NaN,Запрос отменён пользователем из портала самооб...,NaN,2018-02-17 06:04,NaN,1-Особая,4-Нет влияния,Система7,Место325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6110,20913,Разблокировать учетную запись пользователя Сис...,Сервис21,2-Средний,Отменен,ФГ108,2018-03-09 13:42,2018-02-25 12:27,Запрос,NaN,Запрос отменён пользователем из портала самооб...,NaN,2018-02-25 12:36,NaN,1-Особая,4-Нет влияния,Система21,Место42
6128,20931,Консультация или решение проблем связанных с с...,Сервис13,2-Средний,Отменен,ФГ140,2018-03-09 14:01,2018-02-25 12:46,Запрос,NaN,Запрос отменён пользователем из портала самооб...,NaN,2018-02-25 14:27,NaN,1-Особая,3-Малое,Система13,Место24
6216,21019,Отключение учётной записи,Сервис7,2-Средний,Отменен,ФГ13,2018-03-09 15:28,2018-02-25 14:13,Запрос,NaN,Запрос отменён пользователем из портала самооб...,NaN,2018-02-25 14:27,NaN,1-Особая,4-Нет влияния,Система7,Место37
6221,21024,Консультация,Сервис7,2-Средний,Отменен,ФГ43,2018-03-09 15:34,2018-02-25 14:19,Запрос,NaN,Запрос отменён пользователем из портала самооб...,NaN,2018-03-07 15:25,NaN,1-Особая,4-Нет влияния,Система35,Место24


In [36]:
priority = {'3-Низкий': 0, '2-Средний': 1, '1-Высокий': 2, '0-Критический': 3}
status = {'Отменен': 0, 'Закрыт': 1}
obr = {'Запрос': 0, 'Инцидент': 1}
critical = {'4-Нет': 0, "3-Базовая": 2, "2-Повышенная": 3, "1-Особая": 1}
influence = {"4-Нет влияния": 0, "3-Малое": 1, "2-Значительное": 2, "1-Всеохватывающее": 3}

In [37]:
df['Приоритет'] = df['Приоритет'].map(priority)
df

,id,Содержание,Сервис,Приоритет,Статус,Функциональная группа,Крайний срок,Дата обращения,Тип обращения на момент подачи,Тип обращения итоговый,Решение,Тип переклассификации,Дата восстановления,Дата закрытия обращения,Критичность,Влияние,Система,Место
0,14803,Заявка на предоставление и отзыв прав доступа ...,Сервис41,0,Закрыт,ФГ13,2018-02-22 07:51,2018-02-16 13:11,Запрос,NaN,Работы по обращению выполнены,NaN,2018-02-18 04:32,2018-02-25 07:56,4-Нет,4-Нет влияния,Система43,Место38
1,14804,Транспортный шлюз,Сервис61,1,Закрыт,ФГ1,2018-02-21 14:36,2018-02-16 13:13,Запрос,NaN,проинф у.о.,NaN,2018-02-16 17:55,2018-02-24 07:54,1-Особая,3-Малое,Система61,К
2,14805,Заявка на предоставление и отзыв прав доступа ...,Сервис22,0,Закрыт,ФГ13,2018-03-31 06:36,2018-02-16 13:13,Запрос,NaN,Работы по обращению выполнены,NaN,2018-03-15 16:09,2018-03-23 07:56,2-Повышенная,4-Нет влияния,Система13,Место25
3,14806,Заявка на предоставление и отзыв прав доступа ...,Сервис91,0,Отменен,ФГ13,2018-02-21 14:29,2018-02-16 13:14,Запрос,NaN,Запрос отменён пользователем из портала самооб...,NaN,2018-02-16 13:15,NaN,2-Повышенная,4-Нет влияния,Система363,Место1
4,14807,Заявка на предоставление и отзыв прав доступа ...,Сервис22,0,Закрыт,ФГ13,2018-03-30 09:36,2018-02-16 13:14,Запрос,NaN,Работы по обращению выполнены,NaN,2018-03-15 16:09,2018-03-23 07:56,2-Повышенная,4-Нет влияния,Система13,Место25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6339,21142,<Бот-наблюдения> Система53. Ухудшение статуса ...,Сервис53,0,Закрыт,ФГ4,2018-03-07 14:36,2018-02-25 20:56,Инцидент,NaN,самовосстановление,NaN,2018-02-26 05:57,2018-03-14 07:53,2-Повышенная,3-Малое,Система53,Место1
6340,21143,<Бот-наблюдения> Система52. Ухудшение статуса ...,Сервис52,1,Закрыт,ФГ4,2018-03-07 14:06,2018-02-25 20:58,Инцидент,NaN,самовосстановление,NaN,2018-02-26 06:00,2018-03-14 07:53,1-Особая,3-Малое,Система52,Место1
6341,21144,<Бот-наблюдения> Система11. Ухудшение статуса ...,Сервис11,0,Закрыт,ФГ4,2018-03-09 12:36,2018-02-25 21:05,Инцидент,NaN,самовосстановление,NaN,2018-02-26 06:03,2018-03-14 07:53,4-Нет,3-Малое,Система11,Место1
6342,21145,<Бот-наблюдения>Система364. Ухудшение статуса ...,Сервис4,0,Закрыт,ФГ5,2018-03-09 12:36,2018-02-25 21:16,Инцидент,NaN,Самовосстановление,NaN,2018-03-07 09:28,2018-03-14 10:43,2-Повышенная,3-Малое,Система4,Место1


In [38]:
df['Статус'] = df['Статус'].map(status)
df

,id,Содержание,Сервис,Приоритет,Статус,Функциональная группа,Крайний срок,Дата обращения,Тип обращения на момент подачи,Тип обращения итоговый,Решение,Тип переклассификации,Дата восстановления,Дата закрытия обращения,Критичность,Влияние,Система,Место
0,14803,Заявка на предоставление и отзыв прав доступа ...,Сервис41,0,1,ФГ13,2018-02-22 07:51,2018-02-16 13:11,Запрос,NaN,Работы по обращению выполнены,NaN,2018-02-18 04:32,2018-02-25 07:56,4-Нет,4-Нет влияния,Система43,Место38
1,14804,Транспортный шлюз,Сервис61,1,1,ФГ1,2018-02-21 14:36,2018-02-16 13:13,Запрос,NaN,проинф у.о.,NaN,2018-02-16 17:55,2018-02-24 07:54,1-Особая,3-Малое,Система61,К
2,14805,Заявка на предоставление и отзыв прав доступа ...,Сервис22,0,1,ФГ13,2018-03-31 06:36,2018-02-16 13:13,Запрос,NaN,Работы по обращению выполнены,NaN,2018-03-15 16:09,2018-03-23 07:56,2-Повышенная,4-Нет влияния,Система13,Место25
3,14806,Заявка на предоставление и отзыв прав доступа ...,Сервис91,0,0,ФГ13,2018-02-21 14:29,2018-02-16 13:14,Запрос,NaN,Запрос отменён пользователем из портала самооб...,NaN,2018-02-16 13:15,NaN,2-Повышенная,4-Нет влияния,Система363,Место1
4,14807,Заявка на предоставление и отзыв прав доступа ...,Сервис22,0,1,ФГ13,2018-03-30 09:36,2018-02-16 13:14,Запрос,NaN,Работы по обращению выполнены,NaN,2018-03-15 16:09,2018-03-23 07:56,2-Повышенная,4-Нет влияния,Система13,Место25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6339,21142,<Бот-наблюдения> Система53. Ухудшение статуса ...,Сервис53,0,1,ФГ4,2018-03-07 14:36,2018-02-25 20:56,Инцидент,NaN,самовосстановление,NaN,2018-02-26 05:57,2018-03-14 07:53,2-Повышенная,3-Малое,Система53,Место1
6340,21143,<Бот-наблюдения> Система52. Ухудшение статуса ...,Сервис52,1,1,ФГ4,2018-03-07 14:06,2018-02-25 20:58,Инцидент,NaN,самовосстановление,NaN,2018-02-26 06:00,2018-03-14 07:53,1-Особая,3-Малое,Система52,Место1
6341,21144,<Бот-наблюдения> Система11. Ухудшение статуса ...,Сервис11,0,1,ФГ4,2018-03-09 12:36,2018-02-25 21:05,Инцидент,NaN,самовосстановление,NaN,2018-02-26 06:03,2018-03-14 07:53,4-Нет,3-Малое,Система11,Место1
6342,21145,<Бот-наблюдения>Система364. Ухудшение статуса ...,Сервис4,0,1,ФГ5,2018-03-09 12:36,2018-02-25 21:16,Инцидент,NaN,Самовосстановление,NaN,2018-03-07 09:28,2018-03-14 10:43,2-Повышенная,3-Малое,Система4,Место1


In [39]:
df['Тип обращения на момент подачи'] = df['Тип обращения на момент подачи'].map(obr)
df['Тип обращения итоговый'] = df['Тип обращения итоговый'].map(obr)
df

,id,Содержание,Сервис,Приоритет,Статус,Функциональная группа,Крайний срок,Дата обращения,Тип обращения на момент подачи,Тип обращения итоговый,Решение,Тип переклассификации,Дата восстановления,Дата закрытия обращения,Критичность,Влияние,Система,Место
0,14803,Заявка на предоставление и отзыв прав доступа ...,Сервис41,0,1,ФГ13,2018-02-22 07:51,2018-02-16 13:11,0,NaN,Работы по обращению выполнены,NaN,2018-02-18 04:32,2018-02-25 07:56,4-Нет,4-Нет влияния,Система43,Место38
1,14804,Транспортный шлюз,Сервис61,1,1,ФГ1,2018-02-21 14:36,2018-02-16 13:13,0,NaN,проинф у.о.,NaN,2018-02-16 17:55,2018-02-24 07:54,1-Особая,3-Малое,Система61,К
2,14805,Заявка на предоставление и отзыв прав доступа ...,Сервис22,0,1,ФГ13,2018-03-31 06:36,2018-02-16 13:13,0,NaN,Работы по обращению выполнены,NaN,2018-03-15 16:09,2018-03-23 07:56,2-Повышенная,4-Нет влияния,Система13,Место25
3,14806,Заявка на предоставление и отзыв прав доступа ...,Сервис91,0,0,ФГ13,2018-02-21 14:29,2018-02-16 13:14,0,NaN,Запрос отменён пользователем из портала самооб...,NaN,2018-02-16 13:15,NaN,2-Повышенная,4-Нет влияния,Система363,Место1
4,14807,Заявка на предоставление и отзыв прав доступа ...,Сервис22,0,1,ФГ13,2018-03-30 09:36,2018-02-16 13:14,0,NaN,Работы по обращению выполнены,NaN,2018-03-15 16:09,2018-03-23 07:56,2-Повышенная,4-Нет влияния,Система13,Место25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6339,21142,<Бот-наблюдения> Система53. Ухудшение статуса ...,Сервис53,0,1,ФГ4,2018-03-07 14:36,2018-02-25 20:56,1,NaN,самовосстановление,NaN,2018-02-26 05:57,2018-03-14 07:53,2-Повышенная,3-Малое,Система53,Место1
6340,21143,<Бот-наблюдения> Система52. Ухудшение статуса ...,Сервис52,1,1,ФГ4,2018-03-07 14:06,2018-02-25 20:58,1,NaN,самовосстановление,NaN,2018-02-26 06:00,2018-03-14 07:53,1-Особая,3-Малое,Система52,Место1
6341,21144,<Бот-наблюдения> Система11. Ухудшение статуса ...,Сервис11,0,1,ФГ4,2018-03-09 12:36,2018-02-25 21:05,1,NaN,самовосстановление,NaN,2018-02-26 06:03,2018-03-14 07:53,4-Нет,3-Малое,Система11,Место1
6342,21145,<Бот-наблюдения>Система364. Ухудшение статуса ...,Сервис4,0,1,ФГ5,2018-03-09 12:36,2018-02-25 21:16,1,NaN,Самовосстановление,NaN,2018-03-07 09:28,2018-03-14 10:43,2-Повышенная,3-Малое,Система4,Место1


In [40]:
df['Критичность'] = df['Критичность'].map(critical)
df['Влияние'] = df['Влияние'].map(influence)
df

,id,Содержание,Сервис,Приоритет,Статус,Функциональная группа,Крайний срок,Дата обращения,Тип обращения на момент подачи,Тип обращения итоговый,Решение,Тип переклассификации,Дата восстановления,Дата закрытия обращения,Критичность,Влияние,Система,Место
0,14803,Заявка на предоставление и отзыв прав доступа ...,Сервис41,0,1,ФГ13,2018-02-22 07:51,2018-02-16 13:11,0,NaN,Работы по обращению выполнены,NaN,2018-02-18 04:32,2018-02-25 07:56,0,0,Система43,Место38
1,14804,Транспортный шлюз,Сервис61,1,1,ФГ1,2018-02-21 14:36,2018-02-16 13:13,0,NaN,проинф у.о.,NaN,2018-02-16 17:55,2018-02-24 07:54,1,1,Система61,К
2,14805,Заявка на предоставление и отзыв прав доступа ...,Сервис22,0,1,ФГ13,2018-03-31 06:36,2018-02-16 13:13,0,NaN,Работы по обращению выполнены,NaN,2018-03-15 16:09,2018-03-23 07:56,3,0,Система13,Место25
3,14806,Заявка на предоставление и отзыв прав доступа ...,Сервис91,0,0,ФГ13,2018-02-21 14:29,2018-02-16 13:14,0,NaN,Запрос отменён пользователем из портала самооб...,NaN,2018-02-16 13:15,NaN,3,0,Система363,Место1
4,14807,Заявка на предоставление и отзыв прав доступа ...,Сервис22,0,1,ФГ13,2018-03-30 09:36,2018-02-16 13:14,0,NaN,Работы по обращению выполнены,NaN,2018-03-15 16:09,2018-03-23 07:56,3,0,Система13,Место25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6339,21142,<Бот-наблюдения> Система53. Ухудшение статуса ...,Сервис53,0,1,ФГ4,2018-03-07 14:36,2018-02-25 20:56,1,NaN,самовосстановление,NaN,2018-02-26 05:57,2018-03-14 07:53,3,1,Система53,Место1
6340,21143,<Бот-наблюдения> Система52. Ухудшение статуса ...,Сервис52,1,1,ФГ4,2018-03-07 14:06,2018-02-25 20:58,1,NaN,самовосстановление,NaN,2018-02-26 06:00,2018-03-14 07:53,1,1,Система52,Место1
6341,21144,<Бот-наблюдения> Система11. Ухудшение статуса ...,Сервис11,0,1,ФГ4,2018-03-09 12:36,2018-02-25 21:05,1,NaN,самовосстановление,NaN,2018-02-26 06:03,2018-03-14 07:53,0,1,Система11,Место1
6342,21145,<Бот-наблюдения>Система364. Ухудшение статуса ...,Сервис4,0,1,ФГ5,2018-03-09 12:36,2018-02-25 21:16,1,NaN,Самовосстановление,NaN,2018-03-07 09:28,2018-03-14 10:43,3,1,Система4,Место1


In [41]:
len(df)

6344

In [42]:
df['Дата обращения'] = pd.to_datetime(df['Дата обращения'])
df['Дата закрытия обращения'] = pd.to_datetime(df['Дата закрытия обращения'])
df['range_time'] = df['Дата закрытия обращения'] - df['Дата обращения']
df['Крайний срок'] = pd.to_datetime(df['Крайний срок'])
df['time_to_solve'] = (df['Крайний срок'] - df['Дата обращения'])
df

,id,Содержание,Сервис,Приоритет,Статус,Функциональная группа,Крайний срок,Дата обращения,Тип обращения на момент подачи,Тип обращения итоговый,Решение,Тип переклассификации,Дата восстановления,Дата закрытия обращения,Критичность,Влияние,Система,Место,range_time,time_to_solve
0,14803,Заявка на предоставление и отзыв прав доступа ...,Сервис41,0,1,ФГ13,2018-02-22 07:51:00,2018-02-16 13:11:00,0,NaN,Работы по обращению выполнены,NaN,2018-02-18 04:32,2018-02-25 07:56:00,0,0,Система43,Место38,8 days 18:45:00,5 days 18:40:00
1,14804,Транспортный шлюз,Сервис61,1,1,ФГ1,2018-02-21 14:36:00,2018-02-16 13:13:00,0,NaN,проинф у.о.,NaN,2018-02-16 17:55,2018-02-24 07:54:00,1,1,Система61,К,7 days 18:41:00,5 days 01:23:00
2,14805,Заявка на предоставление и отзыв прав доступа ...,Сервис22,0,1,ФГ13,2018-03-31 06:36:00,2018-02-16 13:13:00,0,NaN,Работы по обращению выполнены,NaN,2018-03-15 16:09,2018-03-23 07:56:00,3,0,Система13,Место25,34 days 18:43:00,42 days 17:23:00
3,14806,Заявка на предоставление и отзыв прав доступа ...,Сервис91,0,0,ФГ13,2018-02-21 14:29:00,2018-02-16 13:14:00,0,NaN,Запрос отменён пользователем из портала самооб...,NaN,2018-02-16 13:15,NaT,3,0,Система363,Место1,NaT,5 days 01:15:00
4,14807,Заявка на предоставление и отзыв прав доступа ...,Сервис22,0,1,ФГ13,2018-03-30 09:36:00,2018-02-16 13:14:00,0,NaN,Работы по обращению выполнены,NaN,2018-03-15 16:09,2018-03-23 07:56:00,3,0,Система13,Место25,34 days 18:42:00,41 days 20:22:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6339,21142,<Бот-наблюдения> Система53. Ухудшение статуса ...,Сервис53,0,1,ФГ4,2018-03-07 14:36:00,2018-02-25 20:56:00,1,NaN,самовосстановление,NaN,2018-02-26 05:57,2018-03-14 07:53:00,3,1,Система53,Место1,16 days 10:57:00,9 days 17:40:00
6340,21143,<Бот-наблюдения> Система52. Ухудшение статуса ...,Сервис52,1,1,ФГ4,2018-03-07 14:06:00,2018-02-25 20:58:00,1,NaN,самовосстановление,NaN,2018-02-26 06:00,2018-03-14 07:53:00,1,1,Система52,Место1,16 days 10:55:00,9 days 17:08:00
6341,21144,<Бот-наблюдения> Система11. Ухудшение статуса ...,Сервис11,0,1,ФГ4,2018-03-09 12:36:00,2018-02-25 21:05:00,1,NaN,самовосстановление,NaN,2018-02-26 06:03,2018-03-14 07:53:00,0,1,Система11,Место1,16 days 10:48:00,11 days 15:31:00
6342,21145,<Бот-наблюдения>Система364. Ухудшение статуса ...,Сервис4,0,1,ФГ5,2018-03-09 12:36:00,2018-02-25 21:16:00,1,NaN,Самовосстановление,NaN,2018-03-07 09:28,2018-03-14 10:43:00,3,1,Система4,Место1,16 days 13:27:00,11 days 15:20:00


In [43]:
df['range_time'] = df['range_time'].dt.total_seconds() / 3600
df['time_to_solve'] = df['time_to_solve'].dt.total_seconds() / 3600
df

,id,Содержание,Сервис,Приоритет,Статус,Функциональная группа,Крайний срок,Дата обращения,Тип обращения на момент подачи,Тип обращения итоговый,Решение,Тип переклассификации,Дата восстановления,Дата закрытия обращения,Критичность,Влияние,Система,Место,range_time,time_to_solve
0,14803,Заявка на предоставление и отзыв прав доступа ...,Сервис41,0,1,ФГ13,2018-02-22 07:51:00,2018-02-16 13:11:00,0,NaN,Работы по обращению выполнены,NaN,2018-02-18 04:32,2018-02-25 07:56:00,0,0,Система43,Место38,210.750000,138.666667
1,14804,Транспортный шлюз,Сервис61,1,1,ФГ1,2018-02-21 14:36:00,2018-02-16 13:13:00,0,NaN,проинф у.о.,NaN,2018-02-16 17:55,2018-02-24 07:54:00,1,1,Система61,К,186.683333,121.383333
2,14805,Заявка на предоставление и отзыв прав доступа ...,Сервис22,0,1,ФГ13,2018-03-31 06:36:00,2018-02-16 13:13:00,0,NaN,Работы по обращению выполнены,NaN,2018-03-15 16:09,2018-03-23 07:56:00,3,0,Система13,Место25,834.716667,1025.383333
3,14806,Заявка на предоставление и отзыв прав доступа ...,Сервис91,0,0,ФГ13,2018-02-21 14:29:00,2018-02-16 13:14:00,0,NaN,Запрос отменён пользователем из портала самооб...,NaN,2018-02-16 13:15,NaT,3,0,Система363,Место1,NaN,121.250000
4,14807,Заявка на предоставление и отзыв прав доступа ...,Сервис22,0,1,ФГ13,2018-03-30 09:36:00,2018-02-16 13:14:00,0,NaN,Работы по обращению выполнены,NaN,2018-03-15 16:09,2018-03-23 07:56:00,3,0,Система13,Место25,834.700000,1004.366667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6339,21142,<Бот-наблюдения> Система53. Ухудшение статуса ...,Сервис53,0,1,ФГ4,2018-03-07 14:36:00,2018-02-25 20:56:00,1,NaN,самовосстановление,NaN,2018-02-26 05:57,2018-03-14 07:53:00,3,1,Система53,Место1,394.950000,233.666667
6340,21143,<Бот-наблюдения> Система52. Ухудшение статуса ...,Сервис52,1,1,ФГ4,2018-03-07 14:06:00,2018-02-25 20:58:00,1,NaN,самовосстановление,NaN,2018-02-26 06:00,2018-03-14 07:53:00,1,1,Система52,Место1,394.916667,233.133333
6341,21144,<Бот-наблюдения> Система11. Ухудшение статуса ...,Сервис11,0,1,ФГ4,2018-03-09 12:36:00,2018-02-25 21:05:00,1,NaN,самовосстановление,NaN,2018-02-26 06:03,2018-03-14 07:53:00,0,1,Система11,Место1,394.800000,279.516667
6342,21145,<Бот-наблюдения>Система364. Ухудшение статуса ...,Сервис4,0,1,ФГ5,2018-03-09 12:36:00,2018-02-25 21:16:00,1,NaN,Самовосстановление,NaN,2018-03-07 09:28,2018-03-14 10:43:00,3,1,Система4,Место1,397.450000,279.333333


In [44]:
df['range_time'] = df['range_time'].fillna(-1)

In [45]:
system_l = df['Система'].unique()
system_d = {}
for i in range(len(system_l)):
    system_d[system_l[i]] = i
system_d

{'Система43': 0,
 'Система61': 1,
 'Система13': 2,
 'Система363': 3,
 'База': 4,
 'Система94': 5,
 'Система65': 6,
 'Система22': 7,
 'Система52': 8,
 'Система26': 9,
 'Система11': 10,
 'Система25': 11,
 'Система101': 12,
 'Система7': 13,
 'Система2': 14,
 'Система18': 15,
 'Система23': 16,
 'Система134': 17,
 'Система48': 18,
 'Система245': 19,
 'Система91': 20,
 'Система20': 21,
 'Система44': 22,
 'Система56': 23,
 'Система39': 24,
 'Система126': 25,
 'Система138': 26,
 'Система129': 27,
 'Система38': 28,
 'Система21': 29,
 'Система24': 30,
 'Система12': 31,
 'Система54': 32,
 'Система206': 33,
 'Система108': 34,
 'Система246': 35,
 'Система57': 36,
 'Система64': 37,
 'Система165': 38,
 'Система71': 39,
 'Система31': 40,
 'Система200': 41,
 'Система135': 42,
 'Система35': 43,
 'Система99': 44,
 'Система41': 45,
 'Система131': 46,
 'Система100': 47,
 'Система147': 48,
 'Система10': 49,
 'Система8': 50,
 'Система133': 51,
 'Система309': 52,
 'Система33': 53,
 'Система149': 54,
 'Система

In [46]:
df['Система'] = df['Система'].map(system_d)
df

,id,Содержание,Сервис,Приоритет,Статус,Функциональная группа,Крайний срок,Дата обращения,Тип обращения на момент подачи,Тип обращения итоговый,Решение,Тип переклассификации,Дата восстановления,Дата закрытия обращения,Критичность,Влияние,Система,Место,range_time,time_to_solve
0,14803,Заявка на предоставление и отзыв прав доступа ...,Сервис41,0,1,ФГ13,2018-02-22 07:51:00,2018-02-16 13:11:00,0,NaN,Работы по обращению выполнены,NaN,2018-02-18 04:32,2018-02-25 07:56:00,0,0,0,Место38,210.750000,138.666667
1,14804,Транспортный шлюз,Сервис61,1,1,ФГ1,2018-02-21 14:36:00,2018-02-16 13:13:00,0,NaN,проинф у.о.,NaN,2018-02-16 17:55,2018-02-24 07:54:00,1,1,1,К,186.683333,121.383333
2,14805,Заявка на предоставление и отзыв прав доступа ...,Сервис22,0,1,ФГ13,2018-03-31 06:36:00,2018-02-16 13:13:00,0,NaN,Работы по обращению выполнены,NaN,2018-03-15 16:09,2018-03-23 07:56:00,3,0,2,Место25,834.716667,1025.383333
3,14806,Заявка на предоставление и отзыв прав доступа ...,Сервис91,0,0,ФГ13,2018-02-21 14:29:00,2018-02-16 13:14:00,0,NaN,Запрос отменён пользователем из портала самооб...,NaN,2018-02-16 13:15,NaT,3,0,3,Место1,-1.000000,121.250000
4,14807,Заявка на предоставление и отзыв прав доступа ...,Сервис22,0,1,ФГ13,2018-03-30 09:36:00,2018-02-16 13:14:00,0,NaN,Работы по обращению выполнены,NaN,2018-03-15 16:09,2018-03-23 07:56:00,3,0,2,Место25,834.700000,1004.366667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6339,21142,<Бот-наблюдения> Система53. Ухудшение статуса ...,Сервис53,0,1,ФГ4,2018-03-07 14:36:00,2018-02-25 20:56:00,1,NaN,самовосстановление,NaN,2018-02-26 05:57,2018-03-14 07:53:00,3,1,90,Место1,394.950000,233.666667
6340,21143,<Бот-наблюдения> Система52. Ухудшение статуса ...,Сервис52,1,1,ФГ4,2018-03-07 14:06:00,2018-02-25 20:58:00,1,NaN,самовосстановление,NaN,2018-02-26 06:00,2018-03-14 07:53:00,1,1,8,Место1,394.916667,233.133333
6341,21144,<Бот-наблюдения> Система11. Ухудшение статуса ...,Сервис11,0,1,ФГ4,2018-03-09 12:36:00,2018-02-25 21:05:00,1,NaN,самовосстановление,NaN,2018-02-26 06:03,2018-03-14 07:53:00,0,1,10,Место1,394.800000,279.516667
6342,21145,<Бот-наблюдения>Система364. Ухудшение статуса ...,Сервис4,0,1,ФГ5,2018-03-09 12:36:00,2018-02-25 21:16:00,1,NaN,Самовосстановление,NaN,2018-03-07 09:28,2018-03-14 10:43:00,3,1,108,Место1,397.450000,279.333333


In [47]:
df['koef'] = df['Критичность'] * df['Влияние']
df

,id,Содержание,Сервис,Приоритет,Статус,Функциональная группа,Крайний срок,Дата обращения,Тип обращения на момент подачи,Тип обращения итоговый,...,Тип переклассификации,Дата восстановления,Дата закрытия обращения,Критичность,Влияние,Система,Место,range_time,time_to_solve,koef
0,14803,Заявка на предоставление и отзыв прав доступа ...,Сервис41,0,1,ФГ13,2018-02-22 07:51:00,2018-02-16 13:11:00,0,NaN,...,NaN,2018-02-18 04:32,2018-02-25 07:56:00,0,0,0,Место38,210.750000,138.666667,0
1,14804,Транспортный шлюз,Сервис61,1,1,ФГ1,2018-02-21 14:36:00,2018-02-16 13:13:00,0,NaN,...,NaN,2018-02-16 17:55,2018-02-24 07:54:00,1,1,1,К,186.683333,121.383333,1
2,14805,Заявка на предоставление и отзыв прав доступа ...,Сервис22,0,1,ФГ13,2018-03-31 06:36:00,2018-02-16 13:13:00,0,NaN,...,NaN,2018-03-15 16:09,2018-03-23 07:56:00,3,0,2,Место25,834.716667,1025.383333,0
3,14806,Заявка на предоставление и отзыв прав доступа ...,Сервис91,0,0,ФГ13,2018-02-21 14:29:00,2018-02-16 13:14:00,0,NaN,...,NaN,2018-02-16 13:15,NaT,3,0,3,Место1,-1.000000,121.250000,0
4,14807,Заявка на предоставление и отзыв прав доступа ...,Сервис22,0,1,ФГ13,2018-03-30 09:36:00,2018-02-16 13:14:00,0,NaN,...,NaN,2018-03-15 16:09,2018-03-23 07:56:00,3,0,2,Место25,834.700000,1004.366667,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6339,21142,<Бот-наблюдения> Система53. Ухудшение статуса ...,Сервис53,0,1,ФГ4,2018-03-07 14:36:00,2018-02-25 20:56:00,1,NaN,...,NaN,2018-02-26 05:57,2018-03-14 07:53:00,3,1,90,Место1,394.950000,233.666667,3
6340,21143,<Бот-наблюдения> Система52. Ухудшение статуса ...,Сервис52,1,1,ФГ4,2018-03-07 14:06:00,2018-02-25 20:58:00,1,NaN,...,NaN,2018-02-26 06:00,2018-03-14 07:53:00,1,1,8,Место1,394.916667,233.133333,1
6341,21144,<Бот-наблюдения> Система11. Ухудшение статуса ...,Сервис11,0,1,ФГ4,2018-03-09 12:36:00,2018-02-25 21:05:00,1,NaN,...,NaN,2018-02-26 06:03,2018-03-14 07:53:00,0,1,10,Место1,394.800000,279.516667,0
6342,21145,<Бот-наблюдения>Система364. Ухудшение статуса ...,Сервис4,0,1,ФГ5,2018-03-09 12:36:00,2018-02-25 21:16:00,1,NaN,...,NaN,2018-03-07 09:28,2018-03-14 10:43:00,3,1,108,Место1,397.450000,279.333333,3


In [48]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(df['Содержание'])
df['numeric_sod'] = labels
df

,id,Содержание,Сервис,Приоритет,Статус,Функциональная группа,Крайний срок,Дата обращения,Тип обращения на момент подачи,Тип обращения итоговый,...,Дата восстановления,Дата закрытия обращения,Критичность,Влияние,Система,Место,range_time,time_to_solve,koef,numeric_sod
0,14803,Заявка на предоставление и отзыв прав доступа ...,Сервис41,0,1,ФГ13,2018-02-22 07:51:00,2018-02-16 13:11:00,0,NaN,...,2018-02-18 04:32,2018-02-25 07:56:00,0,0,0,Место38,210.750000,138.666667,0,238
1,14804,Транспортный шлюз,Сервис61,1,1,ФГ1,2018-02-21 14:36:00,2018-02-16 13:13:00,0,NaN,...,2018-02-16 17:55,2018-02-24 07:54:00,1,1,1,К,186.683333,121.383333,1,644
2,14805,Заявка на предоставление и отзыв прав доступа ...,Сервис22,0,1,ФГ13,2018-03-31 06:36:00,2018-02-16 13:13:00,0,NaN,...,2018-03-15 16:09,2018-03-23 07:56:00,3,0,2,Место25,834.716667,1025.383333,0,238
3,14806,Заявка на предоставление и отзыв прав доступа ...,Сервис91,0,0,ФГ13,2018-02-21 14:29:00,2018-02-16 13:14:00,0,NaN,...,2018-02-16 13:15,NaT,3,0,3,Место1,-1.000000,121.250000,0,238
4,14807,Заявка на предоставление и отзыв прав доступа ...,Сервис22,0,1,ФГ13,2018-03-30 09:36:00,2018-02-16 13:14:00,0,NaN,...,2018-03-15 16:09,2018-03-23 07:56:00,3,0,2,Место25,834.700000,1004.366667,0,238
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6339,21142,<Бот-наблюдения> Система53. Ухудшение статуса ...,Сервис53,0,1,ФГ4,2018-03-07 14:36:00,2018-02-25 20:56:00,1,NaN,...,2018-02-26 05:57,2018-03-14 07:53:00,3,1,90,Место1,394.950000,233.666667,3,177
6340,21143,<Бот-наблюдения> Система52. Ухудшение статуса ...,Сервис52,1,1,ФГ4,2018-03-07 14:06:00,2018-02-25 20:58:00,1,NaN,...,2018-02-26 06:00,2018-03-14 07:53:00,1,1,8,Место1,394.916667,233.133333,1,176
6341,21144,<Бот-наблюдения> Система11. Ухудшение статуса ...,Сервис11,0,1,ФГ4,2018-03-09 12:36:00,2018-02-25 21:05:00,1,NaN,...,2018-02-26 06:03,2018-03-14 07:53:00,0,1,10,Место1,394.800000,279.516667,0,164
6342,21145,<Бот-наблюдения>Система364. Ухудшение статуса ...,Сервис4,0,1,ФГ5,2018-03-09 12:36:00,2018-02-25 21:16:00,1,NaN,...,2018-03-07 09:28,2018-03-14 10:43:00,3,1,108,Место1,397.450000,279.333333,3,184


In [49]:
place_l = df['Место'].unique()
place_d = {}
for i in range(len(place_l)):
    place_d[place_l[i]] = i
df['Место'] = df['Место'].map(place_d)
df

,id,Содержание,Сервис,Приоритет,Статус,Функциональная группа,Крайний срок,Дата обращения,Тип обращения на момент подачи,Тип обращения итоговый,...,Дата восстановления,Дата закрытия обращения,Критичность,Влияние,Система,Место,range_time,time_to_solve,koef,numeric_sod
0,14803,Заявка на предоставление и отзыв прав доступа ...,Сервис41,0,1,ФГ13,2018-02-22 07:51:00,2018-02-16 13:11:00,0,NaN,...,2018-02-18 04:32,2018-02-25 07:56:00,0,0,0,0,210.750000,138.666667,0,238
1,14804,Транспортный шлюз,Сервис61,1,1,ФГ1,2018-02-21 14:36:00,2018-02-16 13:13:00,0,NaN,...,2018-02-16 17:55,2018-02-24 07:54:00,1,1,1,1,186.683333,121.383333,1,644
2,14805,Заявка на предоставление и отзыв прав доступа ...,Сервис22,0,1,ФГ13,2018-03-31 06:36:00,2018-02-16 13:13:00,0,NaN,...,2018-03-15 16:09,2018-03-23 07:56:00,3,0,2,2,834.716667,1025.383333,0,238
3,14806,Заявка на предоставление и отзыв прав доступа ...,Сервис91,0,0,ФГ13,2018-02-21 14:29:00,2018-02-16 13:14:00,0,NaN,...,2018-02-16 13:15,NaT,3,0,3,3,-1.000000,121.250000,0,238
4,14807,Заявка на предоставление и отзыв прав доступа ...,Сервис22,0,1,ФГ13,2018-03-30 09:36:00,2018-02-16 13:14:00,0,NaN,...,2018-03-15 16:09,2018-03-23 07:56:00,3,0,2,2,834.700000,1004.366667,0,238
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6339,21142,<Бот-наблюдения> Система53. Ухудшение статуса ...,Сервис53,0,1,ФГ4,2018-03-07 14:36:00,2018-02-25 20:56:00,1,NaN,...,2018-02-26 05:57,2018-03-14 07:53:00,3,1,90,3,394.950000,233.666667,3,177
6340,21143,<Бот-наблюдения> Система52. Ухудшение статуса ...,Сервис52,1,1,ФГ4,2018-03-07 14:06:00,2018-02-25 20:58:00,1,NaN,...,2018-02-26 06:00,2018-03-14 07:53:00,1,1,8,3,394.916667,233.133333,1,176
6341,21144,<Бот-наблюдения> Система11. Ухудшение статуса ...,Сервис11,0,1,ФГ4,2018-03-09 12:36:00,2018-02-25 21:05:00,1,NaN,...,2018-02-26 06:03,2018-03-14 07:53:00,0,1,10,3,394.800000,279.516667,0,164
6342,21145,<Бот-наблюдения>Система364. Ухудшение статуса ...,Сервис4,0,1,ФГ5,2018-03-09 12:36:00,2018-02-25 21:16:00,1,NaN,...,2018-03-07 09:28,2018-03-14 10:43:00,3,1,108,3,397.450000,279.333333,3,184


In [50]:
func_l = df['Функциональная группа'].unique()
func_d = {}
for i in range(len(func_l)):
    func_d[func_l[i]] = i
df['Функциональная группа'] = df['Функциональная группа'].map(func_d)
df

,id,Содержание,Сервис,Приоритет,Статус,Функциональная группа,Крайний срок,Дата обращения,Тип обращения на момент подачи,Тип обращения итоговый,...,Дата восстановления,Дата закрытия обращения,Критичность,Влияние,Система,Место,range_time,time_to_solve,koef,numeric_sod
0,14803,Заявка на предоставление и отзыв прав доступа ...,Сервис41,0,1,0,2018-02-22 07:51:00,2018-02-16 13:11:00,0,NaN,...,2018-02-18 04:32,2018-02-25 07:56:00,0,0,0,0,210.750000,138.666667,0,238
1,14804,Транспортный шлюз,Сервис61,1,1,1,2018-02-21 14:36:00,2018-02-16 13:13:00,0,NaN,...,2018-02-16 17:55,2018-02-24 07:54:00,1,1,1,1,186.683333,121.383333,1,644
2,14805,Заявка на предоставление и отзыв прав доступа ...,Сервис22,0,1,0,2018-03-31 06:36:00,2018-02-16 13:13:00,0,NaN,...,2018-03-15 16:09,2018-03-23 07:56:00,3,0,2,2,834.716667,1025.383333,0,238
3,14806,Заявка на предоставление и отзыв прав доступа ...,Сервис91,0,0,0,2018-02-21 14:29:00,2018-02-16 13:14:00,0,NaN,...,2018-02-16 13:15,NaT,3,0,3,3,-1.000000,121.250000,0,238
4,14807,Заявка на предоставление и отзыв прав доступа ...,Сервис22,0,1,0,2018-03-30 09:36:00,2018-02-16 13:14:00,0,NaN,...,2018-03-15 16:09,2018-03-23 07:56:00,3,0,2,2,834.700000,1004.366667,0,238
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6339,21142,<Бот-наблюдения> Система53. Ухудшение статуса ...,Сервис53,0,1,47,2018-03-07 14:36:00,2018-02-25 20:56:00,1,NaN,...,2018-02-26 05:57,2018-03-14 07:53:00,3,1,90,3,394.950000,233.666667,3,177
6340,21143,<Бот-наблюдения> Система52. Ухудшение статуса ...,Сервис52,1,1,47,2018-03-07 14:06:00,2018-02-25 20:58:00,1,NaN,...,2018-02-26 06:00,2018-03-14 07:53:00,1,1,8,3,394.916667,233.133333,1,176
6341,21144,<Бот-наблюдения> Система11. Ухудшение статуса ...,Сервис11,0,1,47,2018-03-09 12:36:00,2018-02-25 21:05:00,1,NaN,...,2018-02-26 06:03,2018-03-14 07:53:00,0,1,10,3,394.800000,279.516667,0,164
6342,21145,<Бот-наблюдения>Система364. Ухудшение статуса ...,Сервис4,0,1,121,2018-03-09 12:36:00,2018-02-25 21:16:00,1,NaN,...,2018-03-07 09:28,2018-03-14 10:43:00,3,1,108,3,397.450000,279.333333,3,184


In [51]:
label_encoder2 = LabelEncoder()
labels2 = label_encoder2.fit_transform(df[df.columns[9]])
df['numeric_ans'] = labels2
df

,id,Содержание,Сервис,Приоритет,Статус,Функциональная группа,Крайний срок,Дата обращения,Тип обращения на момент подачи,Тип обращения итоговый,...,Дата закрытия обращения,Критичность,Влияние,Система,Место,range_time,time_to_solve,koef,numeric_sod,numeric_ans
0,14803,Заявка на предоставление и отзыв прав доступа ...,Сервис41,0,1,0,2018-02-22 07:51:00,2018-02-16 13:11:00,0,NaN,...,2018-02-25 07:56:00,0,0,0,0,210.750000,138.666667,0,238,0
1,14804,Транспортный шлюз,Сервис61,1,1,1,2018-02-21 14:36:00,2018-02-16 13:13:00,0,NaN,...,2018-02-24 07:54:00,1,1,1,1,186.683333,121.383333,1,644,0
2,14805,Заявка на предоставление и отзыв прав доступа ...,Сервис22,0,1,0,2018-03-31 06:36:00,2018-02-16 13:13:00,0,NaN,...,2018-03-23 07:56:00,3,0,2,2,834.716667,1025.383333,0,238,0
3,14806,Заявка на предоставление и отзыв прав доступа ...,Сервис91,0,0,0,2018-02-21 14:29:00,2018-02-16 13:14:00,0,NaN,...,NaT,3,0,3,3,-1.000000,121.250000,0,238,0
4,14807,Заявка на предоставление и отзыв прав доступа ...,Сервис22,0,1,0,2018-03-30 09:36:00,2018-02-16 13:14:00,0,NaN,...,2018-03-23 07:56:00,3,0,2,2,834.700000,1004.366667,0,238,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6339,21142,<Бот-наблюдения> Система53. Ухудшение статуса ...,Сервис53,0,1,47,2018-03-07 14:36:00,2018-02-25 20:56:00,1,NaN,...,2018-03-14 07:53:00,3,1,90,3,394.950000,233.666667,3,177,0
6340,21143,<Бот-наблюдения> Система52. Ухудшение статуса ...,Сервис52,1,1,47,2018-03-07 14:06:00,2018-02-25 20:58:00,1,NaN,...,2018-03-14 07:53:00,1,1,8,3,394.916667,233.133333,1,176,0
6341,21144,<Бот-наблюдения> Система11. Ухудшение статуса ...,Сервис11,0,1,47,2018-03-09 12:36:00,2018-02-25 21:05:00,1,NaN,...,2018-03-14 07:53:00,0,1,10,3,394.800000,279.516667,0,164,0
6342,21145,<Бот-наблюдения>Система364. Ухудшение статуса ...,Сервис4,0,1,121,2018-03-09 12:36:00,2018-02-25 21:16:00,1,NaN,...,2018-03-14 10:43:00,3,1,108,3,397.450000,279.333333,3,184,0


In [52]:
df.to_csv("DATA/new_test.csv", index=False)

In [ ]:
target = df['Тип обращения итоговый']
features = df[["numeric_sod", "numeric_ans",'Приоритет', 'Тип обращения на момент подачи',
               'Критичность', 'Влияние', 'range_time', 'Система', 'Место',
               'Функциональная группа', 'koef', 'time_to_solve']]
features = sm.add_constant(features)
model = sm.OLS(target, features)
result = model.fit()
model2 = sm.Logit(target, features)
result2 = model2.fit()
print(result.summary())

In [ ]:
print(result2.summary())

In [ ]:
# Визуализация данных, поиск зависимостей
import seaborn as sns

In [ ]:
sns.displot(df, x="Влияние",hue="Тип обращения итоговый", multiple="stack")

In [ ]:
sns.displot(df[df['Влияние']==2 ], x="Приоритет",hue="Тип обращения итоговый", multiple="stack")

In [ ]:
sns.displot(df[df['Влияние']==2 ], x="Критичность",hue="Тип обращения итоговый", multiple="stack")

In [ ]:
df['koef'] = (df['Приоритет'] + df['Критичность'])/df['Влияние']
df

In [ ]:
df[df['Критичность'] == 3]

In [ ]:
sns.displot(df, x="range_time",hue="Тип обращения итоговый", multiple="stack")